In [1]:
'''
TODO:
1. CLEAN locality data (remove others) - Done
2. CLEAN lighting (remove Dusk/Dawn?) - Done
3. CLEAN surfaceCondition(remove others) - Done
4. CLEAN weather (remove Others and cloudy) - Done
5. CLEAN seatBelt(remove Unknown) - done
6. CLEAN age (remove unknown) - done
7. CLEAN gender (remove unknown) - done
8. What to do with Behavior
9. What to do with Distraction
'''

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
import keras
import numpy as np
import ast
# from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
import pandas as pd
import seaborn as sn
import random
random.seed(1234)

/home/vincent/virtualenvironment/markov/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# Hyperparameters
batch_size = 8
time_steps = 61
data_dim = 1
test_split = 0.1
validation_split = 0.1
num_epoch = 30

In [4]:
# Convert json data to dictionary
with open('baseline_data.json') as f:
    data = f.read()
    data_dict_ = ast.literal_eval(data)

In [5]:
# Remove data with less than 61 states
for i in range(4950):
    if len(data_dict_['glance_sequence'][str(i)]) != 61:
        print(i)
        for key in data_dict_.keys():
            data_dict_[key].pop(str(i))

289
506
909
1203
1510
1788
2242
2505
2603
3426
3595
3674
3799
3853
4671


In [21]:
data_dict_train = {}
data_dict_test = {}
data_dict = {}
for key in data_dict_.keys():
#     data_dict_train[key] = np.array([],dtype=np.int32)
#     data_dict_test[key] = np.array([],dtype=np.int32)
    data_dict[key] = []
    data_dict_train[key] = []
    data_dict_test[key] = []

count = -1

# Convert dictionary with index as key into an array
for i in sorted(map(int,data_dict_['id'].keys())): 
    count += 1
    # Traverse through all data columns/keys
    for key in data_dict_.keys():
        # Get the data from the inner dict with the str index as key and append it in order
        data_dict[key].append(data_dict_[key][str(i)])

# for key in data_dict.keys():
#         # Split data according to train-test split, small index used as test data        
#     data_dict_test[key], data_dict_train[key] = cleanData(data_dict,key)
    
#         if count < (test_split*4935):
# #             data_dict_test[key] = np.append(data_dict_test[key],data_dict_[key][str(i)])
#             data_dict_test[key].append(data_dict_[key][str(i)])
#         else:
# #             data_dict_train[key] = np.append(data_dict_train[key],data_dict_[key][str(i)])
#             data_dict_train[key].append(data_dict_[key][str(i)])

# Reshape glance sequences back to the appropriate shape
# data_dict_train['glance_sequence'] = data_dict_train['glance_sequence'].reshape(-1,61,1)
# data_dict_test['glance_sequence'] = data_dict_test['glance_sequence'].reshape(-1,61,1)

In [7]:
xy_trains = []
xy_trains.append((data_dict_train['glance_sequence'],data_dict_train['proximity_to_an_intersection'],'proximity_to_an_interesection',2))
xy_trains.append((data_dict_train['glance_sequence'],data_dict_train['lighting'],'lighting',3))
xy_trains.append((data_dict_train['glance_sequence'],data_dict_train['trafficControl'],'trafficControl',2))
xy_trains.append((data_dict_train['glance_sequence'],data_dict_train['locality'],'locality',4))
xy_trains.append((data_dict_train['glance_sequence'],data_dict_train['trafficDensity'],'trafficDensity',3))
xy_trains.append((data_dict_train['glance_sequence'],data_dict_train['surfaceCondition'],'surfaceCondition',2))
xy_trains.append((data_dict_train['glance_sequence'],data_dict_train['weather'],'weather',2))
xy_trains.append((data_dict_train['glance_sequence'],data_dict_train['alignment'],'alignment',2))
xy_trains.append((data_dict_train['glance_sequence'],data_dict_train['travelLanes'],'travelLanes',2))
xy_trains.append((data_dict_train['glance_sequence'],data_dict_train['trafficDivider'],'trafficDivider',2))
xy_trains.append((data_dict_train['glance_sequence'],data_dict_train['seatBelt'],'seatBelt',2))
xy_trains.append((data_dict_train['glance_sequence'],data_dict_train['age'],'age',3))
xy_trains.append((data_dict_train['glance_sequence'],data_dict_train['gender'],'gender',2))

In [90]:
def cleanData(data_dict, name):
    '''
    return:
        [test_x,test_y],[train_x,train_y]
    '''
    if name == 'proximity_to_an_intersection': # Just split the data accordingly
        glances = data_dict['glance_sequence'] 
        labels = data_dict['proximity_to_an_intersection']
        split = int(test_split*len(labels))
        return [glances[:split],labels[:split]],[glances[split:],labels[split:]]
    elif name == 'lighting':
        cleanedData = []
        glances = data_dict['glance_sequence']
        labels = data_dict['lighting']
        to_keep = [i for i,e in enumerate(labels) if e != 3]
        for i in to_keep:
            cleanedData.append((glances[i],labels[i]))
        random.shuffle(cleanedData)
        split = int(test_split*len(cleanedData))
        return zip(*cleanedData[:split]),zip(*cleanedData[split:])
    elif name == 'trafficControl': # Just split the data accordingly
        glances = data_dict['glance_sequence'] 
        labels = data_dict['trafficControl']
        split = int(test_split*len(labels))
        return [glances[:split],labels[:split]],[glances[split:],labels[split:]]        
    elif name == 'locality':
        cleanedData = []
        glances = data_dict['glance_sequence']
        labels = data_dict['locality']
        to_keep = [i for i,e in enumerate(labels) if e != 4] # Get index that doesn't need to be removed
        for i in to_keep:
            cleanedData.append((glances[i],labels[i]))
        random.shuffle(cleanedData)
        split = int(test_split*len(cleanedData))
        return zip(*cleanedData[:split]),zip(*cleanedData[split:]) # Test, Train
    elif name == 'trafficDensity': # Just split the data accordingly
        glances = data_dict['glance_sequence'] 
        labels = data_dict['trafficDensity']
        split = int(test_split*len(labels))
        return [glances[:split],labels[:split]],[glances[split:],labels[split:]]  
    elif name == 'surfaceCondition':
        cleanedData = []
        glances = data_dict['glance_sequence']
        labels = data_dict['surfaceCondition']
        to_keep = [i for i,e in enumerate(labels) if e != 2] # Get index that doesn't need to be removed
        for i in to_keep:
            cleanedData.append((glances[i],labels[i]))
        random.shuffle(cleanedData)
        split = int(test_split*len(cleanedData))
        return zip(*cleanedData[:split]),zip(*cleanedData[split:]) # Test, Train
    elif name == 'weather':
        cleanedData = []
        glances = data_dict['glance_sequence']
        labels = data_dict['weather']
        to_keep = [i for i,e in enumerate(labels) if e != 3 and e != 2] # Get index that doesn't need to be removed
        for i in to_keep:
            cleanedData.append((glances[i],labels[i]))
        random.shuffle(cleanedData)
        split = int(test_split*len(cleanedData))
        return zip(*cleanedData[:split]),zip(*cleanedData[split:]) # Test, Train
    elif name == 'alignment': # Just split the data accordingly
        glances = data_dict['glance_sequence'] 
        labels = data_dict['alignment']
        split = int(test_split*len(labels))
        return [glances[:split],labels[:split]],[glances[split:],labels[split:]] 
    elif name == 'travelLanes': # Just split the data accordingly
        glances = data_dict['glance_sequence'] 
        labels = data_dict['travelLanes']
        split = int(test_split*len(labels))
        return [glances[:split],labels[:split]],[glances[split:],labels[split:]]
    elif name == 'trafficDivider': # Just split the data accordingly
        glances = data_dict['glance_sequence'] 
        labels = data_dict['trafficDivider']
        split = int(test_split*len(labels))
        return [glances[:split],labels[:split]],[glances[split:],labels[split:]]
    elif name == 'seatBelt':
        cleanedData = []
        glances = data_dict['glance_sequence']
        labels = data_dict['seatBelt']
        to_keep = [i for i,e in enumerate(labels) if e != 2] # Get index that doesn't need to be removed
        for i in to_keep:
            cleanedData.append((glances[i],labels[i]))
        random.shuffle(cleanedData)
        split = int(test_split*len(cleanedData))
        return zip(*cleanedData[:split]),zip(*cleanedData[split:]) # Test, Train
    elif name == 'age':
        cleanedData = []
        glances = data_dict['glance_sequence']
        labels = data_dict['age']
        to_keep = [i for i,e in enumerate(labels) if e != 3] # Get index that doesn't need to be removed
        for i in to_keep:
            cleanedData.append((glances[i],labels[i]))
        random.shuffle(cleanedData)
        split = int(test_split*len(cleanedData))
        return zip(*cleanedData[:split]),zip(*cleanedData[split:]) # Test, Train
    elif name == 'gender':
        cleanedData = []
        glances = data_dict['glance_sequence']
        labels = data_dict['gender']
        to_keep = [i for i,e in enumerate(labels) if e != 2] # Get index that doesn't need to be removed
        for i in to_keep:
            cleanedData.append((glances[i],labels[i]))
        random.shuffle(cleanedData)
        split = int(test_split*len(cleanedData))
        return zip(*cleanedData[:split]),zip(*cleanedData[split:]) # Test, Train
test,train = cleanData(data_dict,'gender')
# Initial data size: 4935
print len(test[1]) + len(train[1])
print len(train[0])
print len(test[0])
# xy_trains.append((data_dict_train['glance_sequence'],data_dict_train['travelLanes'],'travelLanes',2))
# xy_trains.append((data_dict_train['glance_sequence'],data_dict_train['trafficDivider'],'trafficDivider',2))
# xy_trains.append((data_dict_train['glance_sequence'],data_dict_train['seatBelt'],'seatBelt',2))
# xy_trains.append((data_dict_train['glance_sequence'],data_dict_train['age'],'age',3))
# xy_trains.append((data_dict_train['glance_sequence'],data_dict_train['gender'],'gender',2))

4394
3955
439


In [91]:
count = 0
for i in data_dict['age']:
    if i == 3:
        count += 1
print count

541


In [9]:
xy_tests = []
xy_tests.append((data_dict_test['glance_sequence'],data_dict_test['proximity_to_an_intersection'],'proximity_to_an_interesection',2))
xy_tests.append((data_dict_test['glance_sequence'],data_dict_test['lighting'],'lighting',4))
xy_tests.append((data_dict_test['glance_sequence'],data_dict_test['trafficControl'],'trafficControl',2))
xy_tests.append((data_dict_test['glance_sequence'],data_dict_test['locality'],'locality',4))
xy_tests.append((data_dict_test['glance_sequence'],data_dict_test['trafficDensity'],'trafficDensity',3))
xy_tests.append((data_dict_test['glance_sequence'],data_dict_test['surfaceCondition'],'surfaceCondition',2))
xy_tests.append((data_dict_test['glance_sequence'],data_dict_test['weather'],'weather',2))
xy_tests.append((data_dict_test['glance_sequence'],data_dict_test['alignment'],'alignment',2))
xy_tests.append((data_dict_test['glance_sequence'],data_dict_test['travelLanes'],'travelLanes',2))
xy_tests.append((data_dict_test['glance_sequence'],data_dict_test['trafficDivider'],'trafficDivider',2))
xy_tests.append((data_dict_test['glance_sequence'],data_dict_test['seatBelt'],'seatBelt',2))
xy_tests.append((data_dict_test['glance_sequence'],data_dict_test['age'],'age',3))
xy_tests.append((data_dict_test['glance_sequence'],data_dict_test['gender'],'gender',2))

In [10]:
def balanceDataset(x_data, y_data, num_of_class):
    '''
    @args:
        x_data: glance sequence
        y_data: label with values from 0 to n
        num_of_class: num of types of label
    Balance the dataset by oversampling classes with smaller dataset
    In the resulting dataset, each class has size equal to the class with the largest dataset
    '''
    # Array with one inner array for each class
    x_train = [[] for i in range(num_of_class)]
    y_train = [[] for i in range(num_of_class)]
    
    # Allocate data according to class
    for i,y in enumerate(y_data):
        x_train[y].append(x_data[i])
        y_train[y].append(y_data[i])
        
    # Get largest class size
    size = max(map(len,y_train))

    # slice_at = int(0.9*len(y_train))
    
    # Oversample smaller classes randomly
    for x,y in zip(x_train,y_train):
        for i in range(size - len(y)):
            num = random.randint(0,len(y)-1)
            x_data.append(x[num])
            y_data.append(y[num])
    
    # Shuffle data before returning to spread the dataset
    shuffles = zip(x_data,y_data)
    random.shuffle(shuffles)
    x_data,y_data = zip(*shuffles)

    return x_data,y_data

In [11]:
x_train,y_train = balanceDataset(x_train,y_train,2)
# Don't need to balance test set

NameError: name 'x_train' is not defined

In [ ]:
x_train_np =  np.array(x_train).reshape(-1,61,1)
y_train_np =  np.array(y_train)
x_test_np =  np.array(x_test).reshape(-1,61,1)
y_test_np =  np.array(y_test)

In [13]:
def trainAll(xy_trains,xy_tests):
    for trains,tests in zip(xy_trains,xy_tests):
        x_train, y_train = balanceDataset(trains[0],trains[1])
        x_test, y_test = tests[0],tests[1]
        x_train, y_train, x_test, y_test = map(np.array,(x_train,y_train,x_test,y_test))
        if trains[3] <= 2:
            loss='binary_crossentropy'
            metrics='binary_accuracy'
        else:
            loss='categorical_crossentropy'
            metrics='categorical_accuracy'
        train(x_train, y_train, x_test, y_test, loss, metrics, balance_weight=False, name=trains[2])

In [ ]:
def train(x_train_np, y_train_np, x_test_np, y_test_np, loss, metrics, balance_weight, name):
    '''
    @args:
        x_train_np: train glance sequence (numpy array)
        y_train_np: train label (numpy array)
        x_test_np: test glance sequence (numpy array)
        y_test_np: test label (numpy array)
        loss: loss function
        metrics: metrics for accuracy
        class_weight: Should class_weight balancing be done
    '''
    adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False)

    model = Sequential()
    model.add(LSTM(128, dropout=0.0, recurrent_dropout=0.0, input_shape=(time_steps,data_dim), ))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                 optimizer=adam,
                 metrics=['binary_accuracy'])

    print('Train...')
    if balance_weight:
        cw = class_weight.compute_class_weight('balanced', np.unique(y_train_np),y_train_np)
        cw_dict = dict(enumerate(cw))
    else:
        cw_dict = None
    
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', min_delta=0, patience=5, mode='max')
    reduce_LR_on_plateau = keras.callbacks.ReduceLROnPlateau(monitor='val_binary_accuracy', factor=0.1, patience=3, mode='max')
    model_checkpoint = keras.callbacks.ModelCheckpoint("checkpoint_{}".format(name), monitor='val_binary_accuracy', save_best_only=True, mode='max')
    
    model.fit(x_train_np, y_train_np,
             batch_size=batch_size,
             epochs=num_epoch,
             validation_split=validation_split,
             class_weight=cw_dict,
             callbacks = [early_stopping,reduce_LR_on_plateau,model_checkpoint],
             verbose=2)

    predictions_ = model.predict(np.array(x_test_np),batch_size=batch_size)
    predictions = []
    for pred in predictions_:
        if pred < 0.5:
            predictions.append(0)
        else:
            predictions.append(1)

    cm = confusion_matrix(y_test,predictions)
    axis_index = ['Non-junction','Junction']
    axis_predicted = ['Predicted ' + a_i for a_i in axis_index]
    df_cm = pd.DataFrame(cm, index = axis_index, columns = axis_predicted)
    plt.figure(figsize = (10,7))
    sn.heatmap(df_cm,annot=True,cmap='Blues', fmt='g')
    print("accuracy:" ,(cm[0][0] + cm[1][1])/float(len(y_test)))

    score, acc = model.evaluate(x_test_np, y_test_np,
                              batch_size=batch_size)
    print('Test score:', score)
    print('Test accuracy:', acc)

In [ ]:
train(x_train_np, y_train_np, x_test_np, y_test_np, loss='binary_crossentropy', metrics='binary_accuracy', balance_weight=False)

In [ ]:
cw_dict